# MLE Portfolio Project - Customer Churn 

This project is to finish the course `Machine Learning/AI Engineer Career Path` on [CodeCademy](https://www.codecademy.com/) and to showcase what I have learned by implementing some concepts and creating clean and reproducible code.

Dataset: 
I took the data from [Kaggle](https://www.kaggle.com/datasets/blastchar/telco-customer-churn?resource=download) and it is a dataset showing `Telco Customer Churn`. In the following project I will create machine learning pipelines to make out which models perform best and which features are most predictive of a customer churning.

This type of analysis could be of great help to businesses, as it attacks the problem of trend detection and future readiness.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA